# Pandas ile Özet Tablolar: `groupby()` ve `pivot_table()` Rehberi

Bu rehber, Pandas kütüphanesinin en güçlü veri özetleme araçlarından olan `groupby()` ve `pivot_table()`'ı kullanarak nasıl özet tablolar oluşturulacağını anlatır. Bu fonksiyonlar, büyük veri setlerinden hızlı ve etkili bir şekilde anlamlı bilgiler çıkarmak için vazgeçilmezdir.

## 1. Kurulum ve Örnek Veri Seti

Analizlerimizi yapmak için `Project -1` klasöründeki tanıdık çalışan veri setini kullanacağız.

In [ ]:
import pandas as pd
import numpy as np

# Veri setini yükleyelim
file_path = '../Project -1/sirket_calisan_verisi_5000.csv'
df = pd.read_csv(file_path)

# Analiz kolaylığı için eksik verileri basitçe dolduralım
df['maas'].fillna(df['maas'].median(), inplace=True)
df['departman'].fillna(df['departman'].mode()[0], inplace=True)

print("Veri Setinin İlk 5 Satırı:")
display(df.head())

## 2. `groupby()` ile Veri Gruplama ve Özetleme

`groupby()` metodu, **Böl-Uygula-Birleştir (Split-Apply-Combine)** mantığıyla çalışır. Veriyi belirli kategorilere böler, her bir gruba bir fonksiyon uygular ve sonuçları birleştirir.

- **Böl (Split):** Veriyi, gruplamak istediğiniz sütun(lar)a göre alt kümelere ayırır.
- **Uygula (Apply):** Her bir gruba bir veya daha fazla agregasyon (toplama, ortalama, sayma vb.) fonksiyonu uygular.
- **Birleştir (Combine):** Sonuçları yeni bir DataFrame veya Series olarak bir araya getirir.

### a. Tek Sütuna Göre Gruplama

In [ ]:
# Departmanlara göre ortalama maaşı hesaplayalım
ortalama_maas_departman = df.groupby('departman')['maas'].mean()
print("Departmanlara Göre Ortalama Maaş:")
display(ortalama_maas_departman.round(2))

### b. Birden Fazla Sütuna Göre Gruplama

In [ ]:
# Departman ve şehir kırılımında ortalama maaş ve deneyim yılını bulalım
grup_coklu = df.groupby(['departman', 'sehir'])[['maas', 'deneyim_yili']].mean()
print("Departman ve Şehir Kırılımında Ortalama Maaş ve Deneyim:")
display(grup_coklu.round(2))

### c. `.agg()` ile Çoklu Agregasyon Fonksiyonları Uygulama

`.agg()` metodu, her grup için birden fazla özet istatistiği aynı anda hesaplamamızı sağlar.

In [ ]:
# Departmanlara göre maaşın ortalamasını, toplamını ve çalışan sayısını bulalım
agg_sonuc = df.groupby('departman')['maas'].agg(['mean', 'sum', 'count'])
print("Departmanlara Göre Maaş İstatistikleri:")
display(agg_sonuc.round(2))

## 3. `pivot_table()` ile Esnek Özet Tablolar

`pivot_table()`, `groupby()`'a benzer bir işlev görse de, veriyi Excel'deki pivot tablolara çok benzer bir formatta yeniden şekillendirmek için daha esnek bir arayüz sunar.

Temel parametreleri:
- `values`: Özetlenecek (hesaplanacak) değerleri içeren sütun.
- `index`: Sonuç tablosunun satırları olacak sütun.
- `columns`: Sonuç tablosunun sütunları olacak sütun.
- `aggfunc`: Uygulanacak agregasyon fonksiyonu (varsayılan: `mean`).

### a. Temel Pivot Tablo

In [ ]:
# Departmanların (index) ortalama maaşını (values) gösteren bir pivot tablo
pivot_basic = pd.pivot_table(df, values='maas', index='departman', aggfunc='mean')
print("Departmanlara Göre Ortalama Maaş (Pivot Table):")
display(pivot_basic.round(2))

### b. `index` ve `columns` Kullanarak Pivot Tablo

In [ ]:
# Satırlarda departman, sütunlarda cinsiyet olacak şekilde ortalama maaşları gösterelim
pivot_complex = pd.pivot_table(df, values='maas', index='departman', columns='cinsiyet', aggfunc='mean')
print("Departman ve Cinsiyet Kırılımında Ortalama Maaşlar:")
display(pivot_complex.round(2))

### c. Çoklu Agregasyon ve `margins` Parametresi

Birden fazla istatistik hesaplayabilir ve `margins=True` ile ara toplamları (All) ekleyebiliriz.

In [ ]:
# Departman ve cinsiyet kırılımında ortalama ve maksimum maaşı, ara toplamlarla birlikte gösterelim
pivot_advanced = pd.pivot_table(
    df, 
    values='maas', 
    index='departman', 
    columns='cinsiyet', 
    aggfunc=['mean', 'max'],
    margins=True, # Ara toplamları (All) ekler
    fill_value=0    # NaN değerleri 0 ile doldurur
)

print("Gelişmiş Pivot Tablo (Ortalama ve Maksimum Maaş):")
display(pivot_advanced.round(2))

## 4. `crosstab()` ile Frekans Tabloları

`pd.crosstab()`, iki veya daha fazla faktörün frekans tablosunu (çapraz tablo) oluşturmak için özel olarak tasarlanmış bir fonksiyondur. Özellikle kategorik değişkenler arasındaki ilişkiyi görmek için kullanışlıdır.

In [ ]:
# Departman ve şehirdeki çalışan sayılarının frekans tablosu
cross_table = pd.crosstab(df['departman'], df['sehir'])
print("Departman ve Şehir Kırılımında Çalışan Sayısı:")
display(cross_table)

# Yüzdesel dağılımı görmek için normalize=True kullanılabilir
cross_table_percent = pd.crosstab(df['departman'], df['sehir'], normalize='index')
print("
Departman ve Şehir Kırılımında Çalışan Yüzdesi (Satırlara Göre):")
display((cross_table_percent * 100).round(2))

## Sonuç: `groupby` vs. `pivot_table`

- **`groupby()`**: Veriyi gruplara ayırıp üzerinde esnek hesaplamalar yapmak için temel ve güçlü bir araçtır. Sonucu genellikle çok seviyeli bir Series veya DataFrame'dir.
- **`pivot_table()`**: `groupby()` üzerine kurulmuş, özellikle veriyi satır ve sütunlara göre yeniden şekillendirerek (reshaping) 2 boyutlu, okunması kolay özet tablolar oluşturmak için tasarlanmış bir fonksiyondur.

Genellikle, eğer istediğiniz çıktı Excel'deki bir pivot tabloya benziyorsa, `pivot_table()` daha doğrudan bir çözüm sunar. Daha karmaşık, çok adımlı agregasyonlar için `groupby()` daha fazla esneklik sağlar.